<a href="https://colab.research.google.com/github/jansoe/Inno/blob/main/TextGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformers

In [1]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.3 MB/s eta 0:00:00


In [2]:
import transformers
import torch
import textwrap 

import plotly.express as px

## Input : Tokens

In [3]:
model_version = 'EleutherAI/gpt-neo-125m'
#model_version = 'dbmdz/german-gpt2'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_version)

In [4]:
tokenizer.tokenize('This is a test of the tokenizer')

['This', 'Ġis', 'Ġa', 'Ġtest', 'Ġof', 'Ġthe', 'Ġtoken', 'izer']

In [5]:
tokens = tokenizer('This is a test of the tokenizer')['input_ids']
tokens

[1212, 318, 257, 1332, 286, 262, 11241, 7509]

In [6]:
[tokenizer.convert_ids_to_tokens(t) for t in tokens]

['This', 'Ġis', 'Ġa', 'Ġtest', 'Ġof', 'Ġthe', 'Ġtoken', 'izer']

In [7]:
tokenizer.decode(tokens)

'This is a test of the tokenizer'

In [8]:
tokenizer.tokenize('n0thing is corect')

['n', '0', 'thing', 'Ġis', 'Ġcore', 'ct']

In [9]:
tokenizer.vocab

{'Ġlanding': 9581,
 'game': 6057,
 'Ġcontacts': 13961,
 'ĠSiren': 43904,
 'hari': 49573,
 'emphasis': 36663,
 'ĠWAY': 34882,
 'ĠLead': 20116,
 'Ġintroduced': 5495,
 'Ġtainted': 36912,
 'Ġdisgu': 17360,
 'Ġshenanigans': 48949,
 'Ġhig': 1880,
 'ĠLump': 41499,
 'ĠFlickr': 27085,
 'ĠKlux': 45092,
 'Ġacknowled': 7333,
 'ĠLewis': 10174,
 'ĠDT': 24311,
 'Ġdeliberations': 45863,
 'Ġapplies': 8991,
 'Ġthumb': 15683,
 'Ġsteak': 26320,
 'Ġtragedy': 13574,
 'Ġdistorted': 26987,
 'Ġphotograp': 10079,
 'Ġsacred': 13626,
 'ĠStock': 10500,
 'Â¯Â¯Â¯Â¯Â¯Â¯Â¯Â¯Â¯Â¯Â¯Â¯Â¯Â¯Â¯Â¯': 27006,
 'struct': 7249,
 'Clark': 43250,
 'uk': 2724,
 'Ġdifferent': 1180,
 'ĠShutterstock': 37356,
 'tons': 27288,
 'laughs': 28124,
 'Touch': 35211,
 'Ġsymb': 5307,
 'Ġstair': 18761,
 'Brexit': 40730,
 'Ġinaccur': 47283,
 'Rating': 29321,
 'Ġgastrointestinal': 40887,
 'Ġheaded': 9153,
 'CHECK': 50084,
 'Ġdeleg': 8570,
 'PET': 47731,
 'Ġmonstrous': 29100,
 'ĠCourage': 37106,
 '100': 3064,
 'Ġseniors': 23481,
 'ĠBRE': 29377,
 'ĠC

### Test of different tokenizer

Mögliche Modelle: 
* https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads
* https://huggingface.co/models?pipeline_tag=text-classification&sort=downloads

In [10]:
model_name = 'bert-base-uncased' #

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [11]:
tokenizer.tokenize('!!! Test sentence !!!') 

['!', '!', '!', 'test', 'sentence', '!', '!', '!']

## Output

In [12]:
model_version = 'EleutherAI/gpt-neo-125m'

tokenizer = transformers.AutoTokenizer.from_pretrained(model_version)
model = transformers.AutoModelForCausalLM.from_pretrained(model_version)
#_ = model.to('cuda') # Modell auf GPU schieben

In [13]:
prompt = "The sun is"
#prompt = "Is Peter a male or female name. Answer: male."


ids = tokenizer(prompt, return_tensors='pt')['input_ids']
ids

tensor([[ 464, 4252,  318]])

In [14]:
with torch.no_grad():
    out = model.forward(ids)

In [15]:
probabilities = torch.softmax(out.logits, 2).numpy().squeeze()

In [16]:
top10 = probabilities[-1].argsort()[:-11:-1]
top10

array([22751,  7396,   262,   257,   503,   287,  4634,  1464,   319,
         991])

In [17]:
probabilities[-1][top10]

array([0.13434376, 0.07112848, 0.02940449, 0.02640192, 0.02544004,
       0.02446919, 0.02341055, 0.02017728, 0.02001574, 0.01899583],
      dtype=float32)

In [18]:
decoded_most_probable = [tokenizer.decode(i) for i in top10]

px.bar(
    x = decoded_most_probable, 
    y = probabilities[-1][top10], 
    title = 'Next token probability'
)

For text generation the model predicts the probability of the next token. But always choosing the most probable token often leads to very repetitive texts. 

To improve text generation qualtiy, different strategies exist:
- `repetition_penalty`: Tokens that have been choosen before are less likely to be choosen again.
- `do_sample`: Not the token with the highest probability is chosen, but randomly a token is chosen reflecting its probability 
  - `temperature`: the higher the temperature, the more likely even low probability tokens are choosen
  - `top_`: do not randomly draw from all tokens but
    - `top_k`: just choose from the k most probable
    - `top_p`: just choose from the top most probable tokens that have a combined probability of p% 

More detailed explanation [here](https://towardsdatascience.com/how-to-sample-from-language-models-682bceb97277).

In [19]:
prompt = 'The sun is'
torch.manual_seed(10) # the seed controls the randomness

output_sequences = model.generate(
    **tokenizer(prompt, return_tensors='pt'),
    max_length = 128, # Wieviele Token sollen maximal generiert werden
    repetition_penalty = None,
    do_sample = False,
    temperature = 0.7,
    top_k = None, # set to None to ignore or 0 < k <= vocab_size
    top_p = None, # set to None to ignore or 0 < p <= 1
    pad_token_id = tokenizer.eos_token_id
)

In [20]:
output_sequences

tensor([[  464,  4252,   318, 22751,   319,   262,   995,    11,   290,   262,
          5788,   389, 22751,   319,   262,   995,    13,   198,   198,   464,
          4252,   318, 22751,   319,   262,   995,    11,   290,   262,  5788,
           389, 22751,   319,   262,   995,    13,   198,   198,   464,  4252,
           318, 22751,   319,   262,   995,    11,   290,   262,  5788,   389,
         22751,   319,   262,   995,    13,   198,   198,   464,  4252,   318,
         22751,   319,   262,   995,    11,   290,   262,  5788,   389, 22751,
           319,   262,   995,    13,   198,   198,   464,  4252,   318, 22751,
           319,   262,   995,    11,   290,   262,  5788,   389, 22751,   319,
           262,   995,    13,   198,   198,   464,  4252,   318, 22751,   319,
           262,   995,    11,   290,   262,  5788,   389, 22751,   319,   262,
           995,    13,   198,   198,   464,  4252,   318, 22751,   319,   262,
           995,    11,   290,   262,  5788,   389, 2

In [21]:
decoded = tokenizer.decode(output_sequences[0])
print(textwrap.fill(decoded, width = 80, replace_whitespace=False))

The sun is shining on the world, and the stars are shining on the world.

The
sun is shining on the world, and the stars are shining on the world.

The sun is
shining on the world, and the stars are shining on the world.

The sun is
shining on the world, and the stars are shining on the world.

The sun is
shining on the world, and the stars are shining on the world.

The sun is
shining on the world, and the stars are shining on the world.

The sun is
shining on the world, and the stars are shining on


### Task: Parameter Exploration

Experiment with GPT-2 for text generation. How to obtain "good" texts? 


- What do you observe with  `repetition_penalty = None` and `do_sample = False`?
- What happens with `do_sample = True` und different values of `temperature`?
  - add now some values for `top_k`. What changes? 
- What happens, if you start with `prompt_text = '<|endoftext|>'`?